In [ ]:
import pandas as pd
import numpy as np
import sys, gc

In [ ]:
df_tcgabiolinks = pd.read_csv("../TCGA_biolinks.csv", index_col=0).dropna(how="all", axis=1)
df_tcgabiolinks = df_tcgabiolinks[df_tcgabiolinks["cancer.type"]=="COAD"].set_index("pan.samplesID")
df_tcgabiolinks

In [ ]:
df_files=pd.read_csv("files_tcga.dat", index_col=0)
df_files=df_files.reset_index().drop_duplicates("cases.0.submitter_id").set_index("cases.0.submitter_id")
df_files = df_files.reindex(index=df_tcgabiolinks.index, copy=False)
df_files = df_files.transpose().append(df_tcgabiolinks.transpose()).transpose()
df_files = df_files.reset_index().set_index("file_name").fillna("unknown")

bins = np.linspace(0,5,num=4)
cutted = pd.cut(df_files["cases.0.samples.0.longest_dimension"].replace("unknown", np.nan).dropna().astype(float),bins=bins, labels=bins[1:])
df_files["longest_dimension"]=df_files["cases.0.samples.0.longest_dimension"]
df_files["longest_dimension"][cutted.index]=cutted

In [ ]:
df_files.to_csv("files.dat", index=True, header=True)

In [ ]:
df = pd.read_csv("mainTable_all.csv",index_col=0)

In [ ]:
df.columns[~df.columns.isin(df_files.index)]

In [ ]:
df_files[~df_files.index.isin(df.columns)]

In [ ]:
df_files["Subtype_Selected"].unique()

In [ ]:
df = df.reindex(columns=df.columns[df.columns.isin(df_files.index)])
df.to_csv("mainTable.csv", index=True, header=True)
df_files.fillna("unknown").to_csv("files.dat", index=True)

In [ ]:
import scanpy as sc
adata = sc.AnnData(X=df.transpose(), obs=df_files.reindex(index=df.columns))
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=3000)
sc.pl.highly_variable_genes(adata)
hvg = adata.var[adata.var["highly_variable"]==True].index

In [ ]:
df.reindex(index=hvg).to_csv("mainTable.csv", index=True, header=True)

In [ ]:
sys.path.append("/home/jovyan/work/phd/hsbm-occam/")
from sbmtm import sbmtm

In [ ]:
hsbm = sbmtm()

In [ ]:
hsbm.make_graph_from_BoW_df(df.reindex(index=hvg))

In [ ]:
hsbm.save_graph("graph.xml.gz")

In [ ]:
del hsbm
gc.collect()

In [ ]:
df.info()